# Chapter 2: Predictive Inference

In labor economics an important question is what determines the wage of workers. This is a causal question, but we could begin to investigate from a predictive perspective.

In the following wage example, $Y$ is the hourly wage of a worker and $X$ is a vector of worker's characteristics, e.g., education, experience, gender.
Two main questions here are:    

* How to use job-relevant characteristics, such as education and experience, to best predict wages?

* What is the difference in predicted wages between men and women with the same job-relevant characteristics?

In this lab, we focus on the prediction question first.

## 2.1 Data

The data set we consider is from the March Supplement of the U.S. Current Population Survey, year 2015. We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week during at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors; individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below $3$.

The variable of interest $Y$ is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single (never married) workers. The final sample is of size $n = 5150$.

## 2.2 Data Analysis

### 2.2.1 Julia code

In [11]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Dates")
Pkg.add("Plots")
using CSV
using DataFrames
using Dates
using Plots
using Statistics, RData  #upload data of R format 
using CategoricalArrays # categorical data 

   Resolving package versions...
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Manifest.toml`


In [41]:
rdata_read = load("../../../data/wage2015_subsample_inference.RData")
data = rdata_read["data"]
names(data)
println("Number of Rows : ", size(data)[1],"\n","Number of Columns : ", size(data)[2],) #rows and columns

Number of Rows : 5150
Number of Columns : 20


***Variable description***

- occ : occupational classification
- ind : industry classification
- lwage : log hourly wage
- sex : gender (1 female) (0 male)
- shs : some high school
- hsg : High school graduated
- scl : Some College
- clg: College Graduate
- ad: Advanced Degree
- ne: Northeast
- mw: Midwest
- so: South
- we: West
- exp1: experience

In [42]:
[eltype(col) for col = eachcol(data)]

20-element Vector{DataType}:
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 Float64
 CategoricalValue{String, UInt16}
 CategoricalValue{String, UInt8}
 CategoricalValue{String, UInt8}
 CategoricalValue{String, UInt8}

In [43]:
# first 10 lines of the data
first(data,10)

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0
6,11.7308,2.46222,1.0,0.0,0.0,0.0,1.0,0.0,0.0
7,19.2308,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0
8,19.2308,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,12.0,2.48491,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [44]:
#a quick decribe of the data
describe(data)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,wage,23.4104,3.02198,19.2308,528.846,0,Float64
2,lwage,2.97079,1.10591,2.95651,6.2707,0,Float64
3,sex,0.444466,0.0,0.0,1.0,0,Float64
4,shs,0.023301,0.0,0.0,1.0,0,Float64
5,hsg,0.243883,0.0,0.0,1.0,0,Float64
6,scl,0.278058,0.0,0.0,1.0,0,Float64
7,clg,0.31767,0.0,0.0,1.0,0,Float64
8,ad,0.137087,0.0,0.0,1.0,0,Float64
9,mw,0.259612,0.0,0.0,1.0,0,Float64


We are constructing the output variable $Y$ and the matrix $Z$ which includes the characteristics of workers that are given in the data.

In [50]:
n = size(data)[1]
z = select(data, Not([:lwage, :wage]))
p = size(z)[2] 

println("Number of observations : ", n, "\n","Number of raw regressors:", p )

Number of observations : 5150
Number of raw regressors:18


For the outcome variable wage and a subset of the raw regressors, we calculate the empirical mean to get familiar with the data.

In [52]:
z_subset = select(data, ["lwage","sex","shs","hsg","scl","clg","ad","mw","so","we","ne","exp1"])
describe(z_subset, :mean)

,variable,mean
,Symbol,Float64
1,lwage,2.97079
2,sex,0.444466
3,shs,0.023301
4,hsg,0.243883
5,scl,0.278058
6,clg,0.31767
7,ad,0.137087
8,mw,0.259612
9,so,0.296505


# Prediction Question


Now, we will construct a prediction rule for hourly wage $Y$ , which depends linearly on job-relevant characteristics  $X$:

$$Y = \beta' X + \epsilon $$
 
Our goals are

* Predict wages using various characteristics of workers.

* Assess the predictive performance using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample $MSE$ and $R^2$.

We employ two different specifications for prediction:

- **Basic Model**: $X$ consists of a set of raw regressors (e.g. gender, experience, education indicators, occupation and industry indicators, regional indicators).

- **Flexible Model**: $X$ consists of all raw regressors from the basic model plus occupation and industry indicators, transformations (e.g.,$exp2$ and $exp3$) and additional two-way interactions of polynomial in experience with other regressors. An example of a regressor created through a two-way interaction is experience times the indicator of having a college degree.

Using the **Flexible Model**, enables us to approximate the real relationship by a more complex regression model and therefore to reduce the bias. The **Flexible Model** increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver good prediction accuracy but give models which are harder to interpret.

Now, let us fit both models to our data by running ordinary least squares (ols):

In [53]:
#Needed Packages and extra just in case
Pkg.add("Plots")
Pkg.add("Lathe")
Pkg.add("GLM")
Pkg.add("StatsPlots")
Pkg.add("MLBase")

# Load the installed packages
using DataFrames
using CSV
using Plots
using Lathe
using GLM
using Statistics
using StatsPlots
using MLBase

   Resolving package versions...
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Roberto Carlos\.julia\environments\v1.7\Manifest.toml`


In [54]:
#basic model
basic  = @formula(lwage ~ (sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2))
basic_results  = lm(basic, data)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

lwage ~ 1 + sex + exp1 + shs + hsg + scl + clg + mw + so + we + occ2 + ind2

Coefficients:
────────────────────────────────────────────────────────────────────────────────
                  Coef.   Std. Error       t  Pr(>|t|)    Lower 95%    Upper 95%
────────────────────────────────────────────────────────────────────────────────
(Intercept)   3.72224    0.0803414     46.33    <1e-99   3.56473      3.87974
sex          -0.0728575  0.0150269     -4.85    <1e-05  -0.102317    -0.0433983
exp1          0.0085677  0.000653744   13.11    <1e-37   0.00728608   0.00984932
shs          -0.592798   0.0505549    -11.73    <1e-30  -0.691908    -0.493689
hsg          -0.504337   0.0270767    -18.63    <1e-74  -0.557419    -0.451255
scl          -0.411994   0.0252036    -16.35    <1e-57  -0.461404    -0.362584
clg         

In [55]:
#flexible model
flex = @formula(lwage ~ sex + shs+hsg+scl+clg+occ2+ind2+mw+so+we + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we))
regflex = lm(flex, data)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

lwage ~ 1 + sex + shs + hsg + scl + clg + occ2 + ind2 + mw + so + we + exp1 + exp2 + exp3 + exp4 + exp1 & shs + exp1 & hsg + exp1 & scl + exp1 & clg + exp1 & occ2 + exp1 & ind2 + exp1 & mw + exp1 & so + exp1 & we + exp2 & shs + exp2 & hsg + exp2 & scl + exp2 & clg + exp2 & occ2 + exp2 & ind2 + exp2 & mw + exp2 & so + exp2 & we + exp3 & shs + exp3 & hsg + exp3 & scl + exp3 & clg + exp3 & occ2 + exp3 & ind2 + exp3 & mw + exp3 & so + exp3 & we + exp4 & shs + exp4 & hsg + exp4 & scl + exp4 & clg + exp4 & occ2 + exp4 & ind2 + exp4 & mw + exp4 & so + exp4 & we

Coefficients:
────────────────────────────────────────────────────────────────────────────────────
                        Coef.  Std. Error      t  Pr(>|t|)    Lower 95%    Upper 95%
────────────────────────────────────────────────────────────────────────────